Đến thư mục chính. 

In [1]:
ROOT='/content'
%cd $ROOT

/content


# Bước 1 - Cài đặt CenterNet
- Vào link github của [CenterNet](https://github.com/xingyizhou/CenterNet)
- Đi sâu vào đọc [hướng dẫn cài đặt](https://github.com/xingyizhou/CenterNet/blob/master/readme/INSTALL.md) của link trên
- Để việc cài đặt nhanh hơn thì em sử dụng source đã config của CenterNet.

##Step-1 Cài torch & torchvision

In [0]:
!pip install -U torch==1.4 torchvision==0.5 -f https://download.pytorch.org/whl/cu101/torch_stable.html\n

Tất cả đều ngon lành! Vậy là xong Step1

##Step-2 Cài COCOAPI

In [0]:
COCOAPI = '/content/cocoapi'
!git clone https://github.com/cocodataset/cocoapi.git $COCOAPI
%cd $COCOAPI/PythonAPI

In [0]:
%%capture
!make

In [0]:
%%capture
!python setup.py install --user

## Step-3 Clone Repo của CenterNet & Cài những thứ cần thiết

Nếu cài trên máy thì cần cài Cython trước

In [0]:
%cd $ROOT
CenterNet_ROOT = '/content/CenterNet'
!git clone https://github.com/xuan-vy-nguyen/CenterNet.git $CenterNet_ROOT
%cd $CenterNet_ROOT
!pip install -r requirements.txt

In [7]:
%cd $CenterNet_ROOT/src/lib/external

/content/CenterNet/src/lib/external


In [0]:
%%capture
!python setup.py build_ext --inplace

##Step-4 Build DCNv2

In [0]:
%cd $CenterNet_ROOT/src/lib/models/networks
!rm -rf DCNv2
!git clone https://github.com/CharlesShang/DCNv2
%cd DCNv2
!python setup.py build develop

In [0]:
!./make.sh

# Bước 2 - Chỉnh tham số để nhận Dataset & các chi tiết khác

- Bước này chỉ áp dụng cho các source sử dụng code của tác giả gốc [CenterNet](https://github.com/xingyizhou/CenterNet).
- Để cho nhanh thì có thể xài repo của em đã chỉnh sẵn mà chạy thôi (chỉ áp dụng cho AI City Challenge).
- Các dataset khác thì phải đọc [Document](https://docs.google.com/document/d/15YOLLdwbuWffrNhg0BsR6aDBmBMS3Kh2J4y-Y0AKBpE/edit) này (của Phạm Trọng Thắng), nội dung ghi rất rõ cách cài đặt CenterNet & cài đặt tham số để nhận dataset.


# Bước 3 - Tải Dataset

- Link tải dataset [đây](https://drive.google.com/open?id=1xFcfOEfAXjjzdrZbH3glOxv0rHIne8H7)
- Link tải label cho dataset [đây](https://drive.google.com/file/d/1SWsjrSNaRp3CVe9h3Fu41ezkvXcGPy0_/view?usp=sharing)

1. Mọi người tạo trong CenterNet/data thư mục abc, abc/images chỉ chứa toàn ảnh, label chỉ chứa 2 file label.
2. Sau khi tạo xong thì CenterNet/data nên có cấu trúc dạng như sau: 
- abc
- + images
- + + (1000 ảnh)
- + label
- + + instances_val_car.json
- + + instances_train_car.json

Do drive của em có sẵn nên chỉ cần copy qua :v

In [28]:
!mkdir '/content/CenterNet/data/abc/images'
!mkdir '/content/CenterNet/data/abc/labels'
%cd '/content/CenterNet/data/abc'
!cp -r '/content/drive/My Drive/detector/CenterNet/data/abc/images' .

mkdir: cannot create directory ‘/content/CenterNet/data/abc/images’: File exists
/content/CenterNet/data/abc


In [33]:
# kiểm tra xem đủ ảnh trong folder chưa
import os, os.path
# path joining version for other paths
DIR = '/content/CenterNet/data/abc/images'
print(len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

1000


# Bước 4 - Training

Tham số:
- exp_id: thư mục lưu log và chưa model.pth
- arch: kiến trúc backbone mà mình sẽ sử dụng (AI City Challenge thì nên xài dla34).
- batch_size: số dữ liệu trong 1 batch (nếu tràn RAM thì giảm cái này xuống).
- num_workder: số luồng load dữ liệu lên RAM/GPU.
- num_epochs: số lần train dữ liệu.

In [0]:
%cd $CenterNet_ROOT/src
!python main.py ctdet --exp_id abc_dla_34 --arch dla_34 --batch_size 32 --num_workers 2 --num_epochs 2

/content/CenterNet/src
Fix size testing.
training chunk_sizes: [32]
The output will be saved to  /content/CenterNet/src/lib/../../exp/ctdet/abc_dla_34
heads {'hm': 3, 'wh': 2, 'reg': 2}
Namespace(K=100, aggr_weight=0.0, agnostic_ex=False, arch='dla_34', aug_ddd=0.5, aug_rot=0, batch_size=32, cat_spec_wh=False, center_thresh=0.1, chunk_sizes=[32], data_dir='/content/CenterNet/src/lib/../../data', dataset='coco', debug=0, debug_dir='/content/CenterNet/src/lib/../../exp/ctdet/abc_dla_34/debug', debugger_theme='white', demo='', dense_hp=False, dense_wh=False, dep_weight=1, dim_weight=1, down_ratio=4, eval_oracle_dep=False, eval_oracle_hm=False, eval_oracle_hmhp=False, eval_oracle_hp_offset=False, eval_oracle_kps=False, eval_oracle_offset=False, eval_oracle_wh=False, exp_dir='/content/CenterNet/src/lib/../../exp/ctdet', exp_id='abc_dla_34', fix_res=True, flip=0.5, flip_test=False, gpus=[0], gpus_str='0', head_conv=256, heads={'hm': 3, 'wh': 2, 'reg': 2}, hide_data_time=False, hm_hp=True, hm

# Bước 5 - Testing
Dùng để xem IoU các các thông số của model vừa train được trên tập dataset.

- exp_id - trỏ đến thư mục chứa model. Nên = exp_id ở trên nếu xài chính model đó để test.
- arch: kiến trúc backbone
- load_model: đường dẫn đến model

In [64]:
!python test.py ctdet --exp_id abc_dla_34 --arch dla_34 --load_model ../exp/ctdet/abc_dla_34/model_best.pth --trainval

Fix size testing.
training chunk_sizes: [32]
The output will be saved to  /content/CenterNet/src/lib/../../exp/ctdet/abc_dla_34
heads {'hm': 3, 'wh': 2, 'reg': 2}
Namespace(K=100, aggr_weight=0.0, agnostic_ex=False, arch='dla_34', aug_ddd=0.5, aug_rot=0, batch_size=32, cat_spec_wh=False, center_thresh=0.1, chunk_sizes=[32], data_dir='/content/CenterNet/src/lib/../../data', dataset='coco', debug=0, debug_dir='/content/CenterNet/src/lib/../../exp/ctdet/abc_dla_34/debug', debugger_theme='white', demo='', dense_hp=False, dense_wh=False, dep_weight=1, dim_weight=1, down_ratio=4, eval_oracle_dep=False, eval_oracle_hm=False, eval_oracle_hmhp=False, eval_oracle_hp_offset=False, eval_oracle_kps=False, eval_oracle_offset=False, eval_oracle_wh=False, exp_dir='/content/CenterNet/src/lib/../../exp/ctdet', exp_id='abc_dla_34', fix_res=True, flip=0.5, flip_test=False, gpus=[0], gpus_str='0', head_conv=256, heads={'hm': 3, 'wh': 2, 'reg': 2}, hide_data_time=False, hm_hp=True, hm_hp_weight=1, hm_weight

# Bước 5 - Inference

Tạo thư mục input chứa video cần boundingbox, sau đó đẩy hết video cần inference vào đó (lưu ý, nên là chấm duối .mp4, nếu khác thì vào demo_video.py để sửa lại).

Dataset của AICity Challenge track1 được lưu ở [đây](https://drive.google.com/open?id=1-1BmCScH54xz6-fB693EMlWwJU99LUNI), nếu mọi người muốn sử dụng thì đưa nó vào thư mục input

In [11]:
%cd $ROOT
!mkdir input
!cp '/content/drive/My Drive/My_AICity2020/My_Track1/AIC20_track1/Dataset_A/cam_1.mp4' input # em test 1 video cho nó lẹ

/content


Ở bước này mọi người có 2 lựa chọn:
1. Đã tự train -> sử dụng model của mình để inference
2. Chưa train -> sử dụng model của em đã train sẵn để inference.

Đây là [link](https://drive.google.com/file/d/1_e9Aa93B87rYJzzWKlEu9O0ZE8u3DWqg/view?usp=sharing) pretrained model của em, mọi người có thể add nó vào thư mục CenterNet/models (Code bên dưới là do em có sẵn file trong drive nên copy qua cho lẹ).

In [15]:
%cd $CenterNet_ROOT/models
!cp '/content/drive/My Drive/My_AICity2020/My_Track1/dla_34_200e.pth' ./

/content/CenterNet/models


Nên xóa đi thư mục chưa kết quả của lần inference trước

In [0]:
!rm -rf $ROOT/CenterNet/visualization/dla_34
!rm -rf $CenterNet_ROOT/Detection/*

Và chạy inference
- Tham số demo dùng để nhập input của mình là gì (dạng folder chứa video).
- Tham số load_model dùng để trỏ đường dẫn đến model cần dùng để inference

In [66]:
%cd  $CenterNet_ROOT/src
!python demo_video.py ctdet --demo /content/input --load_model ../exp/ctdet/abc_dla_34/model_best.pth

/content/CenterNet/src
Fix size testing.
training chunk_sizes: [32]
The output will be saved to  /content/CenterNet/src/lib/../../exp/ctdet/default
heads {'hm': 2, 'wh': 2, 'reg': 2}
*skip_frame =  0
Creating model...
loaded ../exp/ctdet/abc_dla_34/model_best.pth, epoch 5
Skip loading parameter hm.2.weight, required shapetorch.Size([2, 256, 1, 1]), loaded shapetorch.Size([3, 256, 1, 1]). If you see this, your model does not fully load the pre-trained weight. Please make sure you have correctly specified --arch xxx or set the correct --num_classes for your own dataset.
Skip loading parameter hm.2.bias, required shapetorch.Size([2]), loaded shapetorch.Size([3]). If you see this, your model does not fully load the pre-trained weight. Please make sure you have correctly specified --arch xxx or set the correct --num_classes for your own dataset.
  8% 253/3000 [00:13<02:27, 18.64it/s]Traceback (most recent call last):
  File "demo_video.py", line 90, in <module>
  File "demo_video.py", line 

## Kêt quả
0. Thời gian được in ra ở mục liền kề bên trên (do có xuất cả video nên nó sẽ chậm hơn source dùng cho AI City Challenge).
1. Kết quả xuất ra có 2 dạng chính: bounding-box và video.
2. Video được lưu ở thư mục visualization, thực ra nó chỉ dùng cho debug trong AI-City Challenge, vì CenterNet là 1 bước nhỏ trong đó. Muốn tắt việc sinh video để tăng tốc inference thì vào demo_video, tìm và xóa TUẦN TỰ các phần nằm giữa cặp dấu ###.
3. Bounding-box có 2 dạng: .json và .pkl => Muốn sinh .json thì xài demo.py, còn .pkl thì xài demo_video.py.

*Khuyên dùng demo_video.py*